<h1>Lat/Long Conversion Website - </h1>
<h3>Convert Addresses to Lat/Long from input .csv file</h3>
<p>
This is a web app that I have created which:
    
    (i) asks users to upload a csv file to a website 
        - The csv file must contain  a column called 'Address'
    (ii) calculates the lat/long of the addresses provided
    (iii) displays a dataframe containing the lat/long of the given addresses.
    
I am using flash for rendering the website and geocoders to calculate lat/long

The html and css files for the webpage are named:

    (i) templates/upload.html (main page) and 
    (ii) The css stylesheet is called static/main.css

The working app can be seen and used at:
    bourneupload.herokuapp.com
and a screen capture is shown at the bottom of this notebook.
<p>
    
   Mike Bourne

In [ ]:
from flask import Flask,render_template,request
import pandas as pd
from geopy.geocoders import ArcGIS


#Initial the Flask appllication called 'app'
app=Flask(__name__)



#defining the main starting page of the webapp - 'upload.html'
@app.route("/")
def index():
    return render_template("upload.html")

#defining the def 'success' which is used to update the 'upload.html' page
#This is activated when the user clicks UPLOAD 
@app.route("/success", methods=['POST'])
def success(): 
    
    if request.method == 'POST':
        #gather the data from the upload.html page. Asign to a panda dataframe
        file = request.files['file']
        #read the data into a panda dataframe
        df = pd.read_csv(file)

        #See if the file contains an 'Address' column - this is required
        try:
            df = df[['Address']]
            #if so, create an instance of ArcGIS
            nom=ArcGIS()

            #calculate the coordinates of all the data in the Address column
            #assign to a new column called Coordinates
            df['Coordinates']=df['Address'].apply(nom.geocode)  
            
            #seperate out the Latitude and Longitude from the coordinate column
            #then remove the redundant Coordinates column
            df['Latitude']=df['Coordinates'].apply(lambda x: x.latitude)
            df['Long']=df['Coordinates'].apply(lambda x: x.longitude)
            del df['Coordinates']
            
            #finally, re-render the upload.html page with the table included         
            return render_template("upload.html", 
                text_box="file uploaded - see table below" ,
                tables=[df.to_html(classes="table-bordered", header="true")] )
  
        #if no Address column exists, let the user know
        except:
            return render_template("upload.html", 
            text_box="Error - no 'Address' column in csv file")            
        
    # inform user that there is an error with posting
    else: 
        return render_template("upload.html", 
        text_box="Error - no data uploaded")

if __name__ == '__main__':
    app.debug = True
    app.run()

The image belows is a screencapture of the output website:

<img src="output.png">